In [6]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F3 '''
def F3(X):
    f = bn.F3()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 1.03 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 23.95 , C = 992.98 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_3_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_3_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [10]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [11]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20,30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=182760, Thu Apr 16 18:34:32 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.922175592960923e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.822756125341820e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [21151.96797451] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [19359.73724314] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.791139230025783e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   29    551 7.061812317592761e+03 1.1e+00 1.94e+00  2e+00  2e+00 0:03.4
   64   1216 6.791256720014244e+02 1.1e+00 1.65e+00  2e+00  2e+00 0:07.5
  100   1900 -4.034948770478561e+03 1.2e+00 1.23e+00  1e+00  1e+00 0:11.8
  148   2812 -1.282683489690791e+04 1.2e+00 8.76e-01  9e-01  9e-01 0:17.9
  200   3800 -2.203181907813997e+04 1.2e+00 5.18e-01  5e-01  5e-01 0:24.3
  264   5016 -2.668909426624189e+04 1.2e+00 2.73e-01  3e-01  3e-01 0:32.4
  300   5700 -2.743170028128819e+04 1.2e+00 1.91e-01  2e-01  2e-01 0:37.0
  380   7220 -2.832779420054009e+04 1.2e+00 8.30e-02  8e-02  8e-02 0:47.0
  400   7600 -2.841200697796611e+04 1.2e+00 6.76e-02  6e-02  7e-02 0:49.6
  494   9386 -2.851147944693986e+04 1.3e+00 2.68e-02  3e-02  3e-02 1:01.6
  500   9500 -2.851358382706506e+04 1.3e+00 2.54e-02  2e-02  3e-02 1:02.4
  600  11400 -2.852683202431331e+04 1.3e+00 8.89e-03  8e-03  9e-03 1:15.1
  700  13300 -2.852876143586189e+04 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [22322.25886811] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Thu Apr 16 18:56:49 2020)
final/bestever f-value = 2.232226e+04 2.232226e+04
incumbent solution: [ 3.58253847 -2.85477963  1.38610034  4.93068754 -4.97475367  0.54271455
  4.01147646  1.55678286 ...]
std deviations: [2.43548733 2.43553791 2.43539234 2.43518417 2.43545135 2.43536902
 2.43587966 2.43553429 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=174583, Thu Apr 16 18:56:49 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.059717843483520e+04 1.0e+00 2.43e+00  2e+00  2e+00 0:00.1
    2     38 1.841841160563094e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

    3     57 1.584919360521060e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   28    532 8.169905081330409e+03 1.1e+00 2.03e+00  2e+00  2e+00 0:03.4
   61   1159 3.303483822762479e+03 1.1e+00 1.79e+00  2e+00  2e+00 0:07.6
  100   1900 1.599157596536403e+03 1.2e+00 1.59e+00  2e+00  2e+00 0:12.5
  149   2831 -5.436479048678335e+03 1.2e+00 1.17e+00  1e+00  1e+00 0:18.6
  200   3800 -1.556073275650538e+04 1.3e+00 7.48e-01  7e-01  8e-01 0:24.9
  265   5035 -2.463688974105443e+04 1.3e+00 4.01e-01  4e-01  4e-01 0:33.0
  300   5700 -2.675828584742554e+04 1.3e+00 2.75e-01  3e-01  3e-01 0:37.3
  381   7239 -2.807392846501923e+04 1.3e+00 1.22e-01  1e-01  1e-01 0:47.4
  400   7600 -2.823509346860435e+04 1.3e+00 1.03e-01  1e-01  1e-01 0:49.8
  496   9424 -2.848229595810019e+04 1.3e+00 3.99e-02  4e-02  4e-02 1:01.9
  500   9500 -2.848461753044630e+04 1.3e+00 3.85e-02  4e-02  4e-02 1:02.4
  600  11400 -2.852446817757060e+04 1.3e+00 1.38e-02  1e-02  1e-02 1:14.9
  700  13300 -2.852841463250254e+04 1.3e+0

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [22322.25886811] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Fri Apr 17 00:29:25 2020)
final/bestever f-value = 2.232226e+04 2.232226e+04
incumbent solution: [-3.7401711  -3.91332057 -0.92986413  3.50578108  2.70858904  3.30586716
  1.61898702 -2.26845177 ...]
std deviations: [2.43631604 2.43630254 2.43595789 2.43622384 2.43644952 2.4361703
 2.43640566 2.43624142 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=208486, Fri Apr 17 00:29:25 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.708763937685880e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.721800961664287e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [17087.63937686] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [21312.29693798] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.582130147098312e+04 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3
   31    589 7.624640068552228e+03 1.1e+00 1.89e+00  2e+00  2e+00 0:03.4
   67   1273 4.888505409171357e+03 1.1e+00 1.67e+00  2e+00  2e+00 0:07.4
  100   1900 3.620699816026048e+03 1.2e+00 1.49e+00  1e+00  1e+00 0:11.1
  154   2926 1.550190366134637e+03 1.2e+00 1.24e+00  1e+00  1e+00 0:17.1
  200   3800 -1.515863194473937e+03 1.2e+00 9.27e-01  9e-01  9e-01 0:22.1
  273   5187 -4.550473013867797e+03 1.3e+00 5.97e-01  6e-01  6e-01 0:30.2
  300   5700 -5.099366515039374e+03 1.3e+00 5.18e-01  5e-01  5e-01 0:33.1
  391   7429 -7.225790750184149e+03 1.3e+00 3.01e-01  3e-01  3e-01 0:43.2
  400   7600 -7.160894805578604e+03 1.3e+00 2.90e-01  3e-01  3e-01 0:44.2
  500   9500 -8.196818219126195e+03 1.3e+00 1.82e-01  2e-01  2e-01 0:55.2
  600  11400 -8.698586861004487e+03 1.4e+00 1.10e-01  1e-01  1e-01 1:06.3
  700  13300 -8.982793199779719e+03 1.5e+00 8.74e-02  8e-02  9e-02 1:17.3
  800  15200 -9.242553194601900e+03 1.9e+00

In [12]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F3_X_Values_second.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F3(X_Values.iloc[i,:200])
    SVM_Fun [i] = F3(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F3(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([4292.73440977]), count=array([1])) 8.126086443428e-05
SVM
ModeResult(mode=array([18273.38179529]), count=array([1])) 3736.6179958468133
ELN
ModeResult(mode=array([9774.41750749]), count=array([1])) 286.4698990923148
